<a href="https://colab.research.google.com/github/sourabhgithubcode/10_python_r_business_problems/blob/master/Data4goodDatasavants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install transformers datasets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch


In [ ]:
# Load the data
data = pd.read_csv('/content/examples.csv')  # Ensure the file path is correct

# Combine question columns into a single 'text' column
question_columns = [f'q{i}' for i in range(1, 23)]
data['text'] = data[question_columns].astype(str).agg(' '.join, axis=1)

# Ensure 'label' column exists and is correctly formatted
LABEL_COLUMN = 'label'
unique_labels = data[LABEL_COLUMN].unique()
print("Unique labels in the dataset:", unique_labels)

# Check if labels need remapping to be within the valid range
if not all(label in range(len(unique_labels)) for label in unique_labels):
    label_mapping = {label: i for i, label in enumerate(sorted(unique_labels))}
    data[LABEL_COLUMN] = data[LABEL_COLUMN].map(label_mapping)
    print("New label mapping:", label_mapping)


Unique labels in the dataset: [3 4 2 6 1 5]
New label mapping: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}


In [ ]:
# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

tokenized_data = dataset.map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
num_labels = len(unique_labels)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,  # Replace with a separate validation set if available
    tokenizer=tokenizer
)


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,1.771955
2,No log,1.756909
3,No log,1.750591


TrainOutput(global_step=21, training_loss=1.7827371869768416, metrics={'train_runtime': 82.3702, 'train_samples_per_second': 1.821, 'train_steps_per_second': 0.255, 'total_flos': 39468075724800.0, 'train_loss': 1.7827371869768416, 'epoch': 3.0})

In [ ]:
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_roberta")
tokenizer.save_pretrained("./fine_tuned_roberta")


Evaluation Results: {'eval_loss': 1.7505909204483032, 'eval_runtime': 1.4366, 'eval_samples_per_second': 34.803, 'eval_steps_per_second': 4.872, 'epoch': 3.0}


('./fine_tuned_roberta/tokenizer_config.json',
 './fine_tuned_roberta/special_tokens_map.json',
 './fine_tuned_roberta/vocab.json',
 './fine_tuned_roberta/merges.txt',
 './fine_tuned_roberta/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer for inference
classifier = pipeline("text-classification", model="./fine_tuned_roberta", tokenizer="./fine_tuned_roberta", device=0 if torch.cuda.is_available() else -1)

# Run a sample prediction
sample_text = "Your sample text here for classification."
prediction = classifier(sample_text)
print("Prediction:", prediction)


Prediction: [{'label': 'LABEL_1', 'score': 0.20554223656654358}]


In [ ]:
# Save the trained model and tokenizer
model.save_pretrained("./fine_tuned_roberta")
tokenizer.save_pretrained("./fine_tuned_roberta")


('./fine_tuned_roberta/tokenizer_config.json',
 './fine_tuned_roberta/special_tokens_map.json',
 './fine_tuned_roberta/vocab.json',
 './fine_tuned_roberta/merges.txt',
 './fine_tuned_roberta/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
classifier = pipeline("text-classification", model="./fine_tuned_roberta", tokenizer="./fine_tuned_roberta", device=0 if torch.cuda.is_available() else -1)

# Make a sample prediction
sample_text = "Your sample text here for classification."
prediction = classifier(sample_text)
print("Prediction:", prediction)


OSError: Incorrect path_or_model_id: './fine_tuned_roberta'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

Token indices sequence length is longer than the specified maximum sequence length for this model (770 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The expanded size of the tensor (770) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [8, 770].  Tensor sizes: [1, 514]

In [ ]:
import pandas as pd

# Load the new data
text_data = pd.read_csv('/content/test.csv')  # Adjust the file path as needed

# Combine question columns into a single 'text' column
question_columns = [f'q{i}' for i in range(1, 23)]
text_data['text'] = text_data[question_columns].astype(str).agg(' '.join, axis=1)


In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer for inference
classifier = pipeline("text-classification", model="./fine_tuned_roberta", tokenizer="./fine_tuned_roberta", device=0 if torch.cuda.is_available() else -1)


In [ ]:
# Run batch predictions
predictions = classifier(text_data['text'].tolist(), batch_size=8, truncation=True, max_length=512)

# Extract the predicted labels from the output
text_data['predicted_label'] = [int(pred['label'].split('_')[-1]) for pred in predictions]  # Extract label as an integer


In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
classifier = pipeline(
    "text-classification",
    model="./fine_tuned_roberta",  # Path to your saved model
    tokenizer="./fine_tuned_roberta",  # Path to your saved tokenizer
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise CPU
)


NameError: name 'torch' is not defined

In [ ]:
# Run batch predictions
predictions = classifier(text_data['text'].tolist(), batch_size=8, truncation=True, max_length=512)

# Extract the predicted labels from the output
text_data['predicted_label'] = [int(pred['label'].split('_')[-1]) for pred in predictions]  # Extract label as an integer


In [ ]:
import pandas as pd
from transformers import pipeline


In [ ]:
# Load the test dataset
test_data = pd.read_csv('/content/test.csv')  # Adjust the path as needed

# Combine question columns into a single 'text' column
question_columns = [f'q{i}' for i in range(1, 23)]
test_data['text'] = test_data[question_columns].astype(str).agg(' '.join, axis=1)


In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
classifier = pipeline(
    "text-classification",
    model="./fine_tuned_roberta",  # Path to your saved model
    tokenizer="./fine_tuned_roberta",  # Path to your saved tokenizer
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise CPU
)


In [ ]:
model_path = "/content/fine_tuned_roberta"  # Replace with the actual path in your environment


In [ ]:
import os
from transformers import pipeline

# Ensure that the model and tokenizer paths are correct
model_path = "./fine_tuned_roberta"  # Replace with the full path if necessary

# Check if the directory exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"The directory '{model_path}' does not exist. Please check the path.")

# Load the fine-tuned model and tokenizer
classifier = pipeline(
    "text-classification",
    model=model_path,  # Correct path to your saved model
    tokenizer=model_path,  # Correct path to your saved tokenizer
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise CPU
)


FileNotFoundError: The directory './fine_tuned_roberta' does not exist. Please check the path.

In [ ]:
# Save the model and tokenizer after training
model.save_pretrained("./fine_tuned_roberta")
tokenizer.save_pretrained("./fine_tuned_roberta")


NameError: name 'model' is not defined

In [ ]:
print(test_data.columns)
print(test_data.head())


Index(['Id', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20',
       'q21', 'q22', 'text'],
      dtype='object')
                                     Id  q1   q2  q3  q4  q5  q6  q7  q8  q9  \
0  a2c83973-8e81-45c9-bb84-37fa8a8b637c   3  NaN   0   1   1   1   4   4   1   
1  78bca2d4-8824-45ed-80c9-72ef0e4389c6   1  6.0   2   2   2   1   0   0   2   
2  69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36   3  2.0   1   2   2   2   2   2   2   
3  d5d506d9-29bb-404d-9061-3db9ae92d711   4  NaN   0   2   1   2   2   0   0   
4  d0796afc-055d-4ea8-9552-336ad0442de5   1  NaN   0   1   1   0   1   4   1   

   ...  q14  q15  q16  q17  q18  q19  q20  \
0  ...    3    1    1    0    0    0    0   
1  ...    1    2    2    2    2    2    2   
2  ...    2    2    2    1    1    2    2   
3  ...    2    0    0    2    3    0    2   
4  ...    3    0    1    1    1    0    0   

                                             

In [ ]:
# Run batch predictions
predictions = classifier(test_data['text'].tolist(), batch_size=8, truncation=True, max_length=512)

# Extract the predicted labels and add them to the DataFrame
test_data['predicted_label'] = [int(pred['label'].split('_')[-1]) for pred in predictions]  # Convert 'LABEL_0' to 0, etc.

# Check if the column was added
print(test_data.head())


                                     Id  q1   q2  q3  q4  q5  q6  q7  q8  q9  \
0  a2c83973-8e81-45c9-bb84-37fa8a8b637c   3  NaN   0   1   1   1   4   4   1   
1  78bca2d4-8824-45ed-80c9-72ef0e4389c6   1  6.0   2   2   2   1   0   0   2   
2  69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36   3  2.0   1   2   2   2   2   2   2   
3  d5d506d9-29bb-404d-9061-3db9ae92d711   4  NaN   0   2   1   2   2   0   0   
4  d0796afc-055d-4ea8-9552-336ad0442de5   1  NaN   0   1   1   0   1   4   1   

   ...  q15  q16  q17  q18  q19  q20  \
0  ...    1    1    0    0    0    0   
1  ...    2    2    2    2    2    2   
2  ...    2    2    1    1    2    2   
3  ...    0    0    2    3    0    2   
4  ...    0    1    1    1    0    0   

                                                 q21  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3  "I had a breakthrough at the weekend seminar d...  

In [ ]:
# Save the predictions to a CSV file for submission
test_data[['Id', 'predicted_label']].to_csv('submission.csv', index=False)


In [ ]:
from google.colab import files

# Download the CSV file
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
evaluation_results = trainer.evaluate()
print("Evaluation on validation set:", evaluation_results)


Evaluation on validation set: {'eval_loss': 1.7505909204483032, 'eval_runtime': 1.585, 'eval_samples_per_second': 31.545, 'eval_steps_per_second': 4.416, 'epoch': 3.0}


In [ ]:
from transformers import TrainingArguments, Trainer

# Adjusted training arguments for improvement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,  # Reduced learning rate for stability
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # Increased epochs for more training iterations
    weight_decay=0.01,  # Regularization to help prevent overfitting
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Choose the best model based on validation loss
    save_total_limit=2  # Limit the number of saved models to save disk space
)

# Re-initialize the trainer with new training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,  # Replace with a separate validation set if available
    tokenizer=tokenizer
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.714025
2,No log,1.661568
3,No log,1.603246
4,No log,1.556189
5,No log,1.539423


TrainOutput(global_step=35, training_loss=1.641676984514509, metrics={'train_runtime': 130.3915, 'train_samples_per_second': 1.917, 'train_steps_per_second': 0.268, 'total_flos': 65780126208000.0, 'train_loss': 1.641676984514509, 'epoch': 5.0})

In [ ]:
import torch
from torch.nn import CrossEntropyLoss

# Calculate class weights if you have imbalanced data
class_counts = data['label'].value_counts().sort_index()
class_weights = torch.tensor([1.0 / count for count in class_counts], dtype=torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')

# Modify the model's loss function
model.config.problem_type = "single_label_classification"
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=num_labels
)

# Define a custom training loop if needed for class weights
def compute_loss_with_weights(logits, labels):
    loss_fct = CrossEntropyLoss(weight=class_weights)
    loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))
    return loss


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Ensure dropout is applied in the model configuration
model.config.hidden_dropout_prob = 0.3  # Adjust dropout probability as needed
model.config.attention_probs_dropout_prob = 0.3


In [ ]:
from sklearn.model_selection import KFold
import numpy as np

# Convert the dataset to a format suitable for cross-validation
kf = KFold(n_splits=5)
split_results = []

for train_index, val_index in kf.split(tokenized_data):
    train_subset = tokenized_data.select(train_index)
    val_subset = tokenized_data.select(val_index)

    # Re-initialize and train the model for each fold
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_subset,
        eval_dataset=val_subset,
        tokenizer=tokenizer
    )
    trainer.train()
    eval_result = trainer.evaluate()
    split_results.append(eval_result['eval_loss'])

# Print cross-validation results
print("Cross-Validation Losses:", split_results)
print("Average Cross-Validation Loss:", np.mean(split_results))


Epoch,Training Loss,Validation Loss
1,No log,1.771750
2,No log,1.773068
3,No log,1.775829
4,No log,1.777310
5,No log,1.778121


Epoch,Training Loss,Validation Loss
1,No log,1.806662
2,No log,1.810426
3,No log,1.812312
4,No log,1.813779
5,No log,1.814811


Epoch,Training Loss,Validation Loss
1,No log,1.704074
2,No log,1.721709
3,No log,1.748579
4,No log,1.764541
5,No log,1.767373


Epoch,Training Loss,Validation Loss
1,No log,1.789386
2,No log,1.788300
3,No log,1.756064
4,No log,1.728298
5,No log,1.716365


Epoch,Training Loss,Validation Loss
1,No log,1.600872
2,No log,1.504342
3,No log,1.459285
4,No log,1.429640
5,No log,1.422754
